In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np

# --- 1. Load all datasets ---
# Using content fetched from previous step 
try:
    df_career_pred = pd.read_csv('/kaggle/input/unprocessedcareerguidancedata/career_pred.csv')
    df_career_compass = pd.read_csv('/kaggle/input/unprocessedcareerguidancedata/career_pred.csv')
    df_career_qa = pd.read_csv('/kaggle/input/unprocessedcareerguidancedata/Career20QA20Dataset.csv')
    df_cs_students = pd.read_csv('/kaggle/input/unprocessedcareerguidancedata/cs_students.csv')
    df_career_dataset_excel_sheet = pd.read_excel('/kaggle/input/unprocessedcareerguidancedata/Career Dataset.xlsx')
    df_dataset9000 = pd.read_csv('/kaggle/input/unprocessedcareerguidancedata/dataset9000.csv')

    print("All datasets loaded successfully.")

except FileNotFoundError as e:
    print(f"Error loading file: {e}. Please ensure all CSV files are in the correct directory.")
    exit() # Exit if essential files are not found

# --- 2. Preprocessing and Standardization ---

# --- 2.1. Standardize Skill and Role Names ---
# This is a critical manual step. You need to inspect unique values across relevant columns
# and define consistent mappings. This is illustrative; actual mappings would be more extensive.

# Example mapping for common skill categories from dataset9000 and career_pred
# You'd expand this greatly after full data exploration
skill_mapping = {
    'Database Fundamentals': 'Database_Skills',
    'Computer Architecture': 'Computer_Architecture_Skills',
    'Distributed Computing Systems': 'Distributed_Computing_Skills',
    'Cyber Security': 'Cyber_Security_Skills',
    'Networking': 'Networking_Skills',
    'Software Development': 'Software_Development_Skills',
    'Programming Skills': 'Programming_Skills', # General programming
    'Project Management': 'Project_Management_Skills',
    'Computer Forensics Fundamentals': 'Computer_Forensics_Skills',
    'Technical Communication': 'Technical_Communication_Skills',
    'AI ML': 'AI_ML_Skills',
    'Software Engineering': 'Software_Engineering_Skills',
    'Business Analysis': 'Business_Analysis_Skills',
    'Communication skills': 'Communication_Skills',
    'Data Science': 'Data_Science_Skills',
    'Troubleshooting skills': 'Troubleshooting_Skills',
    'Graphics Designing': 'Graphics_Designing_Skills',
    # From career_pred.csv - need to handle "percentage in" and direct names
    'Acedamic percentage in Operating Systems': 'Operating_Systems_Skills',
    'percentage in Algorithms': 'Algorithms_Skills',
    'Percentage in Programming Concepts': 'Programming_Skills', # Map to general programming
    'Percentage in Software Engineering': 'Software_Engineering_Skills',
    'Percentage in Computer Networks': 'Networking_Skills',
    'Percentage in Electronics Subjects': 'Electronics_Skills',
    'Percentage in Computer Architecture': 'Computer_Architecture_Skills',
    'Percentage in Mathematics': 'Mathematics_Skills',
    'Percentage in Communication skills': 'Communication_Skills',
    'Logical quotient rating': 'Logical_Reasoning_Skills',
    'coding skills rating': 'Coding_Skills_Rating',
    'public speaking points': 'Public_Speaking_Skills',
    'reading and writing skills': 'Reading_Writing_Skills',
    'memory capability score': 'Memory_Capability_Score',
    # Add other soft skills/traits/experience from career_pred.csv here as new features if applicable
    # e.g., 'Hours working per day', 'hackathons', 'can work long time before system?', etc.
}

# Example mapping for common career roles
role_mapping = {
    'Database Administrator': 'Database_Professional',
    'Hardware Engineer': 'Hardware_Engineer',
    'Application Support Engineer': 'Application_Support_Engineer',
    'Cyber Security Specialist': 'Cyber_Security_Specialist',
    'Networking Engineer': 'Networking_Engineer',
    'Software Developer': 'Software_Developer',
    'API Specialist': 'API_Specialist',
    'Project Manager': 'Project_Manager',
    'Information Security Specialist': 'Information_Security_Specialist',
    'Technical Writer': 'Technical_Writer',
    'AI ML Specialist': 'AI_ML_Specialist',
    'Business Analyst': 'Business_Analyst',
    'Customer Service Executive': 'Customer_Service_Executive',
    'Data Scientist': 'Data_Scientist',
    'Helpdesk Engineer': 'Helpdesk_Engineer',
    'Graphics Designer': 'Graphics_Designer',
    # From career_pred.csv
    'Database Developer': 'Database_Professional',
    'Portal Administrator': 'Administrator', # General admin if specific is not found
    'Systems Security Administrator': 'Cyber_Security_Specialist',
    'Business Systems Analyst': 'Business_Analyst',
    'Mobile Applications Developer': 'Mobile_App_Developer',
    'UX Designer': 'UX_UI_Designer',
    'Quality Assurance Associate': 'QA_Engineer',
    'Web Developer': 'Web_Developer',
    'Information Security Analyst': 'Information_Security_Specialist',
    'CRM Business Analyst': 'Business_Analyst',
    'Business Intelligence Analyst': 'Business_Intelligence_Analyst',
    'Programmer Analyst': 'Programmer_Analyst',
    'Technical Support': 'Technical_Support_Engineer',
    'Design & UX': 'UX_UI_Designer',
    'Solutions Architect': 'Solutions_Architect',
    'Database Manager': 'Database_Professional', # Map to general database if no specific 'manager' role
    'Information Technology Manager': 'IT_Manager',
    'Network Engineer': 'Networking_Engineer',
    'Technical Engineer': 'Technical_Engineer',
    'Software Quality Assurance (QA) / Testing': 'QA_Engineer',
    'Systems Analyst': 'Systems_Analyst',
    'CRM Technical Developer': 'Software_Developer', # Assuming this is a type of dev
    'Technical Services/Help Desk/Tech Support': 'Technical_Support_Engineer',
    'Data Architect': 'Data_Architect',
    'Application Support Engineer': 'Application_Support_Engineer',
    'Product Manager': 'Product_Manager'
}


# --- 2.2. Preprocess dataset9000.csv ---
# Convert categorical interest levels to numerical
interest_level_map = {
    'Not Interested': 0,
    'Poor': 1,
    'Beginner': 2,
    'Average': 3,
    'Intermediate': 4,
    'Excellent': 5,
    'Professional': 6
}
df_dataset9000_processed = df_dataset9000.copy()
for col in df_dataset9000_processed.columns[:-1]: # All columns except 'Role'
    df_dataset9000_processed[col] = df_dataset9000_processed[col].map(interest_level_map)

# Rename columns based on skill_mapping
df_dataset9000_processed = df_dataset9000_processed.rename(columns=skill_mapping)
# Rename 'Role' column for consistency
df_dataset9000_processed = df_dataset9000_processed.rename(columns={'Role': 'Target_Role'})
df_dataset9000_processed['Target_Role'] = df_dataset9000_processed['Target_Role'].map(role_mapping)


# --- 2.3. Preprocess career_pred.csv ---
df_career_pred_processed = df_career_pred.copy()

# Convert percentage columns to numerical (already are, just rename)
rename_cols_pred = {k: v for k, v in skill_mapping.items() if k in df_career_pred_processed.columns}
df_career_pred_processed = df_career_pred_processed.rename(columns=rename_cols_pred)

# Convert other categorical features to numerical (e.g., 'yes'/'no' to 1/0)
binary_map = {'yes': 1, 'no': 0}
for col in ['can work long time before system?', 'self-learning capability?', 'worked in teams ever?', 'Introvert']:
    if col in df_career_pred_processed.columns:
        df_career_pred_processed[col] = df_career_pred_processed[col].map(binary_map)

# Map target role
df_career_pred_processed = df_career_pred_processed.rename(columns={'Suggested Job Role': 'Target_Role'})
df_career_pred_processed['Target_Role'] = df_career_pred_processed['Target_Role'].map(role_mapping)

# Drop potentially problematic or highly unique columns for initial merge if not handled in mapping
cols_to_drop_pred = [
    'Student ID', 'Name', 'Interested subjects', 'interested career area ',
    'Job/Higher Studies?', 'Type of company want to settle in?',
    'Taken inputs from seniors or elders', 'interested in games',
    'Interested Type of Books', 'Salary Range Expected',
    'In a Realtionship?', 'Gentle or Tuff behaviour?', 'Management or Technical',
    'Salary/work', 'hard/smart worker'
]
df_career_pred_processed = df_career_pred_processed.drop(columns=[col for col in cols_to_drop_pred if col in df_career_pred_processed.columns])


# --- 2.4. Preprocess cs_students.csv ---
df_cs_students_processed = df_cs_students.copy()

# Convert skill levels (Python, SQL, Java) to numerical
skill_strength_map = {'Strong': 3, 'Average': 2, 'Weak': 1} # Or 0-1 scale if preferred
df_cs_students_processed['Python_Skills'] = df_cs_students_processed['Python'].map(skill_strength_map)
df_cs_students_processed['SQL_Skills'] = df_cs_students_processed['SQL'].map(skill_strength_map)
df_cs_students_processed['Java_Skills'] = df_cs_students_processed['Java'].map(skill_strength_map)


# Map 'Future Career' to standardized roles
df_cs_students_processed = df_cs_students_processed.rename(columns={'Future Career': 'Target_Role'})
df_cs_students_processed['Target_Role'] = df_cs_students_processed['Target_Role'].map(role_mapping)

# Drop irrelevant columns for the merged analytical dataset
cols_to_drop_students = ['Student ID', 'Name', 'Gender', 'Age', 'Major', 'Interested Domain', 'Projects', 'Python', 'SQL', 'Java']
df_cs_students_processed = df_cs_students_processed.drop(columns=[col for col in cols_to_drop_students if col in df_cs_students_processed.columns])

# Rename GPA to a more generic academic performance score if needed
df_cs_students_processed = df_cs_students_processed.rename(columns={'GPA': 'Academic_Performance_Score'})


# --- 3. Align and Concatenate the Feature-Rich Datasets ---

# Identify all unique features across processed datasets that should be in the final merged df
all_features = set()
for df in [df_dataset9000_processed, df_career_pred_processed, df_cs_students_processed]:
    # Exclude 'Target_Role' and columns clearly used for mapping that are not features themselves
    current_features = [col for col in df.columns if col != 'Target_Role' and not col.endswith('_Skills') and col not in ['GPA', 'Academic_Performance_Score']]
    # Add mapped skill columns
    mapped_skills = [col for col in df.columns if '_Skills' in col or col == 'Academic_Performance_Score' or col == 'Coding_Skills_Rating']
    all_features.update(current_features + mapped_skills)

# Convert to list for consistent ordering
all_features = sorted(list(all_features))

# Add 'Target_Role' at the end
final_columns = all_features + ['Target_Role']

# Function to reindex and fill missing columns for each DataFrame
def align_dataframe_columns(df, target_columns):
    # Separate features and target for alignment
    df_features = df.drop(columns=['Target_Role'], errors='ignore')
    df_target = df['Target_Role'] if 'Target_Role' in df.columns else None

    # Identify common and unique columns after initial processing steps
    aligned_features = {}
    for col in target_columns:
        if col != 'Target_Role': # Don't try to fill target column with features
            if col in df_features.columns:
                aligned_features[col] = df_features[col]
            else:
                # Fill missing numerical features with 0, boolean with 0, and others with NaN/None
                # This needs careful consideration based on data types and meaning
                # For simplicity, filling numeric-like skills/ratings with 0 (Not Interested)
                # For other types, fill with a sensible default or leave as NaN.
                if 'Skills' in col or 'Rating' in col or 'Score' in col:
                    aligned_features[col] = 0 # Assuming 0 for 'Not Interested' in skills
                else:
                    aligned_features[col] = np.nan # For other non-directly mapped numeric features

    # Recreate DataFrame with aligned columns
    aligned_df = pd.DataFrame(aligned_features, columns=[c for c in target_columns if c != 'Target_Role'])

    # Add back the target column if it existed
    if df_target is not None:
        aligned_df['Target_Role'] = df_target

    return aligned_df

# Align all three processed dataframes
aligned_df_dataset9000 = align_dataframe_columns(df_dataset9000_processed, final_columns)
aligned_df_career_pred = align_dataframe_columns(df_career_pred_processed, final_columns)
aligned_df_cs_students = align_dataframe_columns(df_cs_students_processed, final_columns)

# Concatenate them
merged_df_analytical = pd.concat([aligned_df_dataset9000, aligned_df_career_pred, aligned_df_cs_students], ignore_index=True)

# Drop rows where 'Target_Role' is NaN (due to unmatched roles or initial mapping issues)
merged_df_analytical.dropna(subset=['Target_Role'], inplace=True)

print(f"\nMerged Analytical Dataset Shape: {merged_df_analytical.shape}")
print("\nSample of Merged Analytical Dataset:")
print(merged_df_analytical.head())

# --- 4. Role of Q&A and Lookup Datasets ---
print("\n--- Understanding the role of other datasets ---")
print("`Career Dataset.xlsx - Sheet1.csv` is a lookup table mapping careers to skills. It can be used to refine or validate the 'skill_mapping' and 'role_mapping' definitions manually, or to generate synthetic data based on career-skill relationships.") 
print("`CareerCompassDataset.csv` contains Q&A about the Career Compass platform and some role descriptions. This data is primarily textual and would be used for Natural Language Processing (NLP) tasks, like building a conversational AI, or extracting more nuanced textual features (e.g., using embeddings) for the predictive model, rather than direct tabular merging.") 
print("`Career QA Dataset.csv` also contains Q&A about various roles. Similar to `CareerCompassDataset.csv`, it's best utilized for NLP or as a knowledge base, not for direct tabular merging into the numerical prediction dataset.") 
print("\nThese Q&A datasets provide rich contextual information that can enhance the user experience of a 'CS Compass' platform, and could serve as data for fine-tuning large language models for career guidance, but they don't directly merge into the primary numerical dataset for predictive modeling without significant feature engineering (e.g., text embeddings).")

# --- 5. Save the merged dataset to a CSV file for download ---
# The index=False argument prevents pandas from writing the DataFrame index as a column
merged_df_analytical.to_csv('merged_analytical_data.csv', index=False)

print("\nDataset saved successfully to 'merged_analytical_data.csv'.")
print("You can now download it from the output directory.")

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from collections import Counter

# --- 1. Load the Dataset ---
try:
    df = pd.read_csv('/kaggle/working/merged_analytical_data.csv')
    print("Dataset loaded successfully.")
    print(f"Original dataset shape: {df.shape}")
except FileNotFoundError:
    print("Error: 'merged_analytical_data.csv' not found. Please ensure it's in the correct directory.")
    exit()

# --- 2. Prepare Data and Identify Column Types ---

# Separate features (X) and target variable (y)
X = df.drop('Target_Role', axis=1)
y = df['Target_Role']

# Identify numerical and categorical columns
# We will treat columns with dtype 'object' as categorical.
# All other columns (int64, float64) will be treated as numerical.
numerical_features = X.select_dtypes(include=np.number).columns.tolist()
categorical_features = X.select_dtypes(include=['object']).columns.tolist()

print(f"\nIdentified {len(numerical_features)} numerical features.")
print(f"Identified {len(categorical_features)} categorical features.")


# --- 3. Define the Preprocessing Pipelines ---

# Create a pipeline for numerical features
# Step 1: Impute missing values with the median of the column. Median is robust to outliers.
# Step 2: Scale the features to have a mean of 0 and a standard deviation of 1.
numerical_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Create a pipeline for categorical features
# Step 1: Impute missing values with the most frequent value (the mode).
# Step 2: One-hot encode the features. 'handle_unknown='ignore'' prevents errors
#         if a category appears in the test set but not the training set.
categorical_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

# Combine the numerical and categorical pipelines into a single preprocessor
# This object will apply the correct transformation to each column type.
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_pipeline, numerical_features),
        ('cat', categorical_pipeline, categorical_features)
    ],
    remainder='passthrough' # Keep other columns if any (should be none in this case)
)

print("\nPreprocessing pipelines created successfully.")


# --- 4. Split Data and Apply the Full Pipeline with SMOTE ---

# Split the data into training and testing sets BEFORE any preprocessing or resampling
# 'stratify=y' ensures that the distribution of the target variable is the same
# in both the training and testing sets, which is crucial for imbalanced datasets.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"\nData split into training and testing sets.")
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")

# --- Apply the preprocessing ---
# Fit the preprocessor on the training data and transform it
X_train_processed = preprocessor.fit_transform(X_train)

# ONLY transform the test data using the preprocessor fitted on the training data
# This prevents data leakage from the test set.
X_test_processed = preprocessor.transform(X_test)

print("\nPreprocessing applied to training and testing sets.")
print(f"Shape of X_train after preprocessing: {X_train_processed.shape}")
print(f"Shape of X_test after preprocessing: {X_test_processed.shape}")


# --- 5. Address Class Imbalance using SMOTE ---

# IMPORTANT: Apply SMOTE only to the training data to avoid data leakage.
# The model should be tested on a non-oversampled, real-world-like distribution.
smote = SMOTE(random_state=42)
print("\nOriginal training target distribution:")
print(sorted(Counter(y_train).items()))

X_train_resampled, y_train_resampled = smote.fit_resample(X_train_processed, y_train)

print("\nSMOTE applied to the training data.")
print("\nResampled training target distribution:")
print(sorted(Counter(y_train_resampled).items()))

print(f"\nShape of X_train after SMOTE resampling: {X_train_resampled.shape}")


# --- Summary of Final, Model-Ready Datasets ---
print("\n--- Pipeline Complete ---")
print("You now have the following model-ready datasets:")
print(f"1. X_train_resampled: The preprocessed and oversampled training features. Shape: {X_train_resampled.shape}")
print(f"2. y_train_resampled: The oversampled training target. Shape: {y_train_resampled.shape}")
print(f"3. X_test_processed: The preprocessed testing features. Shape: {X_test_processed.shape}")
print(f"4. y_test: The original testing target. Shape: {y_test.shape}")



In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

# --- 1. Load the Dataset ---
try:
    df = pd.read_csv('merged_analytical_data.csv')
    print("Dataset loaded successfully.")
    print(f"Original dataset shape: {df.shape}")
except FileNotFoundError:
    print("Error: 'merged_analytical_data.csv' not found. Please ensure it's in the correct directory.")
    exit()

# --- 2. Prepare Data and Identify Column Types ---
X = df.drop('Target_Role', axis=1)
y = df['Target_Role']

numerical_features = X.select_dtypes(include=np.number).columns.tolist()
categorical_features = X.select_dtypes(include=['object']).columns.tolist()

print(f"\nIdentified {len(numerical_features)} numerical features.")
print(f"Identified {len(categorical_features)} categorical features.")

# --- 3. Define the Preprocessing Pipelines ---
numerical_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_pipeline, numerical_features),
        ('cat', categorical_pipeline, categorical_features)
    ],
    remainder='passthrough'
)

print("\nPreprocessing pipelines created successfully.")

# --- 4. Split Data and Apply Preprocessing ---
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

print("\nPreprocessing applied to training and testing sets.")

# --- 5. Address Class Imbalance using SMOTE ---
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_processed, y_train)

print("\nSMOTE applied to the training data.")
print(f"Shape of X_train after SMOTE: {X_train_resampled.shape}")

# --- 6. Encode Target Variable ---
# XGBoost requires the target variable to be encoded as integers (0, 1, 2, ...).
le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train_resampled)
y_test_encoded = le.transform(y_test)

# --- 7. Train the XGBoost Model on GPU ---
print("\n--- Training XGBoost Model on GPU ---")

# To use GPUs, set device='cuda'. XGBoost will automatically detect and
# utilize all available GPUs in the Kaggle environment.
# 'objective': 'multi:softmax' is used for multi-class classification.
# 'num_class': The number of unique job roles.
xgb_classifier = xgb.XGBClassifier(
    objective='multi:softmax',
    num_class=len(le.classes_),
    device='cuda',  # This is the key parameter for GPU training
    eval_metric='mlogloss',
    use_label_encoder=False,
    random_state=42
)

# Train the model
xgb_classifier.fit(X_train_resampled, y_train_encoded)

print("\nModel training complete.")

# --- 8. Evaluate the Model ---
print("\n--- Model Evaluation ---")

# Make predictions on the test set
y_pred_encoded = xgb_classifier.predict(X_test_processed)

# Calculate accuracy
accuracy = accuracy_score(y_test_encoded, y_pred_encoded)
print(f"Model Accuracy: {accuracy:.4f}")

# Decode the predictions and true labels to get the original role names
y_pred_labels = le.inverse_transform(y_pred_encoded)
y_test_labels = le.inverse_transform(y_test_encoded)

# Display the classification report
print("\nClassification Report:")
print(classification_report(y_test_labels, y_pred_labels))

# Display the confusion matrix
print("\nConfusion Matrix:")
cm = confusion_matrix(y_test_labels, y_pred_labels, labels=le.classes_)
plt.figure(figsize=(16, 12))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=le.classes_, yticklabels=le.classes_)
plt.title('Confusion Matrix')
plt.ylabel('Actual Role')
plt.xlabel('Predicted Role')
plt.show()

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import joblib # Import joblib for saving the model

# --- 1. Load the Dataset ---
try:
    df = pd.read_csv('merged_analytical_data.csv')
    print("Dataset loaded successfully.")
    print(f"Original dataset shape: {df.shape}")
except FileNotFoundError:
    print("Error: 'merged_analytical_data.csv' not found. Please ensure it's in the correct directory.")
    exit()

# --- 2. Prepare Data and Identify Column Types ---
X = df.drop('Target_Role', axis=1)
y = df['Target_Role']

numerical_features = X.select_dtypes(include=np.number).columns.tolist()
categorical_features = X.select_dtypes(include=['object']).columns.tolist()

print(f"\nIdentified {len(numerical_features)} numerical features.")
print(f"Identified {len(categorical_features)} categorical features.")

# --- 3. Define the Preprocessing Pipelines ---
numerical_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_pipeline, numerical_features),
        ('cat', categorical_pipeline, categorical_features)
    ],
    remainder='passthrough'
)

print("\nPreprocessing pipelines created successfully.")

# --- 4. Split Data and Apply Preprocessing ---
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

print("\nPreprocessing applied to training and testing sets.")

# --- 5. Address Class Imbalance using SMOTE ---
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_processed, y_train)

print("\nSMOTE applied to the training data.")
print(f"Shape of X_train after SMOTE: {X_train_resampled.shape}")

# --- 6. Encode Target Variable ---
le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train_resampled)
y_test_encoded = le.transform(y_test)

# --- 7. Hyperparameter Tuning with RandomizedSearchCV ---
print("\n--- Performing Hyperparameter Tuning ---")

# Define the parameter grid to search
# These are some of the most important hyperparameters for XGBoost
param_grid = {
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'n_estimators': [100, 200, 300, 500],
    'max_depth': [3, 5, 7, 10],
    'subsample': [0.7, 0.8, 0.9, 1.0],
    'colsample_bytree': [0.7, 0.8, 0.9, 1.0],
    'gamma': [0, 0.1, 0.2]
}

# Initialize the XGBoost classifier for GPU
xgb_classifier = xgb.XGBClassifier(
    objective='multi:softmax',
    num_class=len(le.classes_),
    device='cuda',
    eval_metric='mlogloss',
    use_label_encoder=False,
    random_state=42
)

# Set up RandomizedSearchCV
# n_iter: Number of parameter settings that are sampled.
# cv: Number of folds for cross-validation.
# verbose=2: Prints updates as it runs.
# n_jobs=-1: Uses all available CPU cores for the search.
random_search = RandomizedSearchCV(
    estimator=xgb_classifier,
    param_distributions=param_grid,
    n_iter=25,  # You can increase this for a more thorough search
    cv=3,
    verbose=2,
    random_state=42,
    n_jobs=-1
)

# Fit the random search to the data
random_search.fit(X_train_resampled, y_train_encoded)

print("\nHyperparameter tuning complete.")
print(f"Best Parameters Found: {random_search.best_params_}")

# Get the best model found by the search
best_model = random_search.best_estimator_

# --- 8. Evaluate the Best Model ---
print("\n--- Evaluating Best Model ---")

# Make predictions on the test set using the best model
y_pred_encoded = best_model.predict(X_test_processed)

# Calculate accuracy
accuracy = accuracy_score(y_test_encoded, y_pred_encoded)
print(f"Tuned Model Accuracy: {accuracy:.4f}")

# Decode the predictions and true labels to get the original role names
y_pred_labels = le.inverse_transform(y_pred_encoded)
y_test_labels = le.inverse_transform(y_test_encoded)

# Display the classification report
print("\nClassification Report:")
print(classification_report(y_test_labels, y_pred_labels))

# Display the confusion matrix
print("\nConfusion Matrix:")
cm = confusion_matrix(y_test_labels, y_pred_labels, labels=le.classes_)
plt.figure(figsize=(16, 12))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=le.classes_, yticklabels=le.classes_)
plt.title('Confusion Matrix for Tuned Model')
plt.ylabel('Actual Role')
plt.xlabel('Predicted Role')
plt.show()

# --- 9. Save the Model and Components for Future Use ---
print("\n--- Saving Model and Components ---")

# Save the trained model
joblib.dump(best_model, 'xgboost_career_model.pkl')
print("Trained model saved to 'xgboost_career_model.pkl'")

# Save the preprocessor
joblib.dump(preprocessor, 'preprocessor.pkl')
print("Preprocessor saved to 'preprocessor.pkl'")

# Save the label encoder
joblib.dump(le, 'label_encoder.pkl')
print("Label encoder saved to 'label_encoder.pkl'")

